In [120]:
import subprocess as sp
import numpy as np
import pandas as pd
from io import StringIO
import os
import re
import shutil

In [121]:
#input and output dirs
datadirs = ["../data"]
outputdir = "."

# Functions

In [122]:
selectkeys = ['Kernel Name', 'Metric Name', 'Metric Type']
resultkeys = ['Kernel Name', 'Calls']

def merge_results(df, name):
    if name+'_x' and name+'_y' in df.columns:
        df[name] = df[name+'_x'] + df[name+'_y']
        del df[name+'_x']
        del df[name+'_y']
        
def transpose_frame(df_results, df_metrics):    
    tc_peak_perf_flops = 125*10**12

    # Cleanup: group metric values
    metricdf = df_metrics.copy()
    metricdf = metricdf.groupby(selectkeys).sum().reset_index()
    metricdf = metricdf[['Kernel Name', 'Calls', 'Metric Name', 'Metric Type', 'Metric Value']]
    metricdf.sort_values(by=resultkeys)
    
    # Raise if data not consistent
    if df_results.empty:
        df_results = metricdf[resultkeys].drop_duplicates().reset_index(drop=True).sort_values(by=resultkeys).copy()
        print(df_results)
    else:
        tmpMetricdf = metricdf[resultkeys].drop_duplicates().reset_index(drop=True).sort_values(by=resultkeys).copy()
        tmpResultdf = df_results[resultkeys].drop_duplicates().reset_index(drop=True).sort_values(by=resultkeys).copy()
        if not tmpMetricdf.equals(tmpResultdf):
            print("\n##### Data not in the output DF")
            df = tmpMetricdf.merge(tmpResultdf, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
            print(df)
            print("##### Data not in the current DF")
            df = tmpResultdf.merge(tmpMetricdf, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
            print(df)
            raise ValueError("Data not consistent")

    metriclist = metricdf['Metric Name'].unique()
    

    ####### Get number of FLOPs
    
    ### FMA FLOPs = number of FMA instructions x 2
    metricdf.loc[metricdf["Metric Name"].str.contains("fma_pred_on"), ["Metric Value"]] *= 2
    
    
    ### FP32 FLOPs
    metrics = ['smsp__sass_thread_inst_executed_op_fadd_pred_on',
               'smsp__sass_thread_inst_executed_op_ffma_pred_on',
               'smsp__sass_thread_inst_executed_op_fmul_pred_on']
    if any(m in metriclist for m in metrics):
        tmpdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
        metricname = 'FP32 FLOPs'
        tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": metricname})
        # Merge current df with the result df
        df_results = df_results.merge(tmpdf, on=resultkeys, how="outer")
        # Sum up if exits and remove duplicates
        merge_results(df_results, metricname)
        #print(df_results)
    
    
    ### FP16 FLOPs
    metrics = ['smsp__sass_thread_inst_executed_op_hadd_pred_on',
               'smsp__sass_thread_inst_executed_op_hfma_pred_on',
               'smsp__sass_thread_inst_executed_op_hmul_pred_on']
    if any(m in metriclist for m in metrics):
        tmpdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
        metricname = 'FP16 FLOPs'
        tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": metricname})
        # Merge current df with the result df
        df_results = df_results.merge(tmpdf, on=resultkeys, how="outer")
        # Sum up if exits and remove duplicates
        merge_results(df_results, metricname)
        #print(df_results)
    
    
    ### TC FLOP Rates
    if any("tensor_op_hmma.avg.pct_of_peak" in m for m in metriclist):
        tmpdf = metricdf.loc[ metricdf["Metric Name"].str.contains("tensor_op_hmma.avg.pct_of_peak"), resultkeys+["Metric Value"] ].reset_index().copy()
        tmpdf["Utilization"] = 0.01 * tmpdf["Metric Value"] / tmpdf['Calls']
        metricname = "TC FLOP Rates"
        tmpdf[metricname] = tc_peak_perf_flops * tmpdf["Utilization"]
        # merge
        df_results = df_results.merge(tmpdf[resultkeys+[metricname]], on=resultkeys, how="outer")
        merge_results(df_results, metricname)
        #print(df_results)
    
    ### Total FLOPs
    #metricdf["FLOPs Avg"] = metricdf["FP32 FLOPs Avg"] + metricdf["FP16 FLOPs Avg"] + metricdf["TC FLOPs Avg"] #+ metricdf["FP64 FLOPs"]

    
    ### FLOPs fractions
    #metricdf["FP64 FLOPs Fraction"] = metricdf["FP64 FLOPs"]/metricdf["FLOPs"]
    #metricdf["FP32 FLOPs Fraction Avg"] = metricdf["FP32 FLOPs Avg"]/metricdf["FLOPs Avg"]
    #metricdf["FP16 FLOPs Fraction Avg"] = metricdf["FP16 FLOPs Avg"]/metricdf["FLOPs Avg"]
    #metricdf["TC FLOPs Fraction Avg"]   = metricdf["TC FLOPs Avg"]/metricdf["FLOPs Avg"]
    #print(metricdf)
    
    

    ####### Get timing information

    ### CUDA Time
    if any("smsp__cycles_elapsed" in m for m in metriclist):
        # get cycles
        metricname = "CUDA Cycles"
        cyclesdf = metricdf.loc[(metricdf["Metric Name"]=="smsp__cycles_elapsed") & (metricdf["Metric Type"]=="total"),
                               resultkeys+["Metric Value"]].reset_index(drop=True).rename(columns={"Metric Value": metricname}).copy()
        df_results = df_results.merge(cyclesdf, on=resultkeys, how="outer")
        # get rates
        metricname = "CUDA Rates"
        ratesdf = metricdf.loc[(metricdf["Metric Name"]=="smsp__cycles_elapsed") & (metricdf["Metric Type"]=="rate"),
                               resultkeys+["Metric Value"]].reset_index(drop=True).rename(columns={"Metric Value": metricname}).copy()        
        df_results = df_results.merge(ratesdf[resultkeys+[metricname]], on=resultkeys, how="outer")
        #print(df_results)
    
    
    ### Tensor Core Time
    if any("smsp__pipe_tensor_op_hmma_cycles_active" in m for m in metriclist):
        # get cycles
        metricname = "TC Cycles"
        cyclesdf = metricdf.loc[(metricdf["Metric Name"]=="smsp__pipe_tensor_op_hmma_cycles_active") & (metricdf["Metric Type"]=="total"),
                               resultkeys+["Metric Value"]].reset_index(drop=True).rename(columns={"Metric Value": metricname}).copy()
        df_results = df_results.merge(cyclesdf, on=resultkeys, how="outer")
        # get rates
        metricname = "TC Rates"
        ratesdf = metricdf.loc[(metricdf["Metric Name"]=="smsp__pipe_tensor_op_hmma_cycles_active") & (metricdf["Metric Type"]=="rate"),
                               resultkeys+["Metric Value"]].reset_index(drop=True).rename(columns={"Metric Value": metricname}).copy()        
        df_results = df_results.merge(ratesdf[resultkeys+[metricname]], on=resultkeys, how="outer")
        #print(df_results)
        


    ####### Get number of bytes

    ### FP32 FLOPs
    metrics = ['smsp__sass_thread_inst_executed_op_fadd_pred_on',
               'smsp__sass_thread_inst_executed_op_ffma_pred_on',
               'smsp__sass_thread_inst_executed_op_fmul_pred_on']
    if any(m in metriclist for m in metrics):
        tmpdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
        metricname = 'FP32 FLOPs'
        tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": metricname})
        # Merge current df with the result df
        df_results = df_results.merge(tmpdf, on=resultkeys, how="outer")
        # Sum up if exits and remove duplicates
        merge_results(df_results, metricname)
        #print(df_results)
    
    
    ### Shared transactions
    #project out
    if any("l1tex__data_pipe_lsu_wavefronts_mem_shared_op" in m for m in metriclist):
        shareddf = metricdf.loc[metricdf["Metric Name"].str.contains("l1tex__data_pipe_lsu_wavefronts_mem_shared_op"), resultkeys+["Metric Value"] ].copy()
        metricname = 'Shared Transactions'
        shareddf = shareddf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": metricname})
        df_results = df_results.merge(shareddf, on=resultkeys, how="outer")
        merge_results(df_results, metricname)

    ### L1 atomic transactions
    # project out
    metrics = ['l1tex__t_set_accesses_pipe_lsu_mem_global_op_atom',
               'l1tex__t_set_accesses_pipe_lsu_mem_global_op_red',
               'l1tex__t_set_accesses_pipe_tex_mem_surface_op_atom',
               'l1tex__t_set_accesses_pipe_tex_mem_surface_op_red']
    if any(m in metriclist for m in metrics):
        tmpdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
        metricname = 'L1 Atomic Transactions'
        tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": metricname})
        # Merge current df with the result df
        df_results = df_results.merge(tmpdf, on=resultkeys, how="outer")
        # Sum up if exits and remove duplicates
        merge_results(df_results, metricname)
        #print(df_results)
        
    ### Local transactions
    #project out
    if any("l1tex__t_sectors_pipe_lsu_mem_local_op" in m for m in metriclist):
        localdf = metricdf.loc[metricdf["Metric Name"].str.contains("l1tex__t_sectors_pipe_lsu_mem_local_op"), resultkeys+["Metric Value"] ].copy()
        metricname = 'Local Transactions'
        localdf = localdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": metricname})
        df_results = df_results.merge(localdf, on=resultkeys, how="outer")
        merge_results(df_results, metricname)
        
    return df_results
    
    ### Global transactions 
    # project out
    globaldf = metricdf.loc[metricdf["Metric Name"].str.contains("l1tex__t_sectors_pipe_lsu_mem_global_op"), selectkeys+["Metric Value"] ].copy()
    globaldf = globaldf.groupby(selectkeys).sum().reset_index().rename(columns={"Metric Value": "Global Transactions Avg"})
    # add to timings
    metricdf = metricdf.merge(globaldf[selectkeys+["Global Transactions Avg"]], on=selectkeys, how="inner")
    
    
    ### L1 Bytes
    metricdf["L1 Transactions Avg"] = (metricdf["Shared Transactions Avg"] + metricdf["L1 Atomic Transactions Avg"]
                            + metricdf["Local Transactions Avg"] + metricdf["Global Transactions Avg"])
    metricdf["L1 Bytes Avg"] = metricdf["L1 Transactions Avg"] * 32
    
    # clean up
    #del metricdf["Shared Transactions Avg"]
    #del metricdf["L1 Atomic Transactions Avg"]
    #del metricdf["Local Transactions Avg"]
    #del metricdf["Global Transactions Avg"]
    
    
    ### L2 atomic & reduction
    metricdf.loc[(metricdf["Metric Name"].str.contains("lts__t_sectors_op")) & (metricdf["Metric Type"]=="total"), ["Metric Value"]] *= 2
    
    
    ### L2 transactions
    # project out
    l2df = metricdf.loc[metricdf["Metric Name"].str.contains("lts__t_sectors_op"), selectkeys+["Metric Value"] ].copy()
    l2df = l2df.groupby(selectkeys).sum().reset_index().rename(columns={"Metric Value": "L2 Transactions Avg"})
    l2df["L2 Bytes Avg"] = l2df["L2 Transactions Avg"] * 32
    # add to timings
    metricdf = metricdf.merge(l2df[selectkeys+["L2 Transactions Avg", "L2 Bytes Avg"]], on=selectkeys, how="inner")
    
    
    ### DRAM Bytes
    # project out
    dramdf = metricdf[ metricdf["Metric Name"].str.contains("dram__sectors") ].sort_values(selectkeys)
    # get reads and writes
    dramreadsdf = dramdf.loc[(dramdf["Metric Name"]=="dram__sectors") & (dramdf["Metric Type"]=="read"), selectkeys+["Metric Value"]]
    dramwritesdf = dramdf.loc[(dramdf["Metric Name"]=="dram__sectors") & (dramdf["Metric Type"]=="write"), selectkeys+["Metric Value"]]
    # combine
    dramdf = dramwritesdf.merge(dramreadsdf, on=selectkeys, how="outer").fillna(0.)
    dramdf["DRAM Transactions Avg"] = dramdf["Metric Value_x"] + dramdf["Metric Value_y"]
    dramdf["DRAM Bytes Avg"] = dramdf["DRAM Transactions Avg"] * 32
    #print(dramdf[['Name', 'Metric Value_x', 'Metric Value_y']])
    metricdf = metricdf.merge(dramdf[selectkeys+["DRAM Transactions Avg", "DRAM Bytes Avg"]], on=selectkeys, how="inner")
    
    
    
    ####### Clean up and return:
    del metricdf["Metric Value"]
    del metricdf["Metric Name"]
    del metricdf["Metric Type"]
    #del metricdf["Invocations"]
    metricdf.drop_duplicates(keep = 'first', inplace = True)
    

    ### Get performance
    metricdf["Performance GFlop/s"]      = metricdf["FLOPs Avg"]      / (metricdf["CUDA Time Avg"]*10**9)
    metricdf["FP32 Performance GFlop/s"] = metricdf["FP32 FLOPs Avg"] / (metricdf["CUDA Time Avg"]*10**9)
    metricdf["FP16 Performance GFlop/s"] = metricdf["FP16 FLOPs Avg"] / (metricdf["CUDA Time Avg"]*10**9)
    metricdf["TC Performance GFlop/s"]   = metricdf["TC FLOPs Avg"]   / (metricdf["TC Time Avg"]*10**9)

    
    ### Get AI
    # L1
    metricdf["L1 AI"]        = metricdf["FLOPs Avg"]      / metricdf["L1 Bytes Avg"]
    metricdf["FP32 L1 AI"]   = metricdf["FP32 FLOPs Avg"] / metricdf["L1 Bytes Avg"]
    metricdf["FP16 L1 AI"]   = metricdf["FP16 FLOPs Avg"] / metricdf["L1 Bytes Avg"]
    metricdf["TC L1 AI"]     = metricdf["TC FLOPs Avg"]   / metricdf["L1 Bytes Avg"]
    # L2
    metricdf["L2 AI"]        = metricdf["FLOPs Avg"]      / metricdf["L2 Bytes Avg"]
    metricdf["FP32 L2 AI"]   = metricdf["FP32 FLOPs Avg"] / metricdf["L2 Bytes Avg"]
    metricdf["FP16 L2 AI"]   = metricdf["FP16 FLOPs Avg"] / metricdf["L2 Bytes Avg"]
    metricdf["TC L2 AI"]     = metricdf["TC FLOPs Avg"]   / metricdf["L2 Bytes Avg"]
    # DRAM
    metricdf["DRAM AI"]      = metricdf["FLOPs Avg"]      / metricdf["DRAM Bytes Avg"]
    metricdf["FP32 DRAM AI"] = metricdf["FP32 FLOPs Avg"] / metricdf["DRAM Bytes Avg"]
    metricdf["FP16 DRAM AI"] = metricdf["FP16 FLOPs Avg"] / metricdf["DRAM Bytes Avg"]
    metricdf["TC DRAM AI"]   = metricdf["TC FLOPs Avg"]   / metricdf["DRAM Bytes Avg"]


    ### Cleanup
    metricdf.sort_values(by=selectkeys).reset_index(drop=True, inplace=True)
    #print(metricdf[['CUDA Time Avg', 'TC Time Avg']])
    
    return metricdf

# Import Data

In [123]:
#get all the files
files = []
for datadir in datadirs:
    files += [ os.path.join(datadir,x) for x in os.listdir(datadir) if ((os.path.splitext(x)[-1] == ".log"))]

#recs
records = []

#build feature list:
for path in files:
    
    #filename
    file = os.path.basename(path)
    
    #path
    path = os.path.dirname(path)
    
    #splitup
    splt = file.split(".")
    
    prefix = ".".join(splt[0:-1])
    
    #append to records
    records.append({"prefix": prefix, "file": os.path.join(path, file)})

#put in df
recorddf = pd.DataFrame(records).sort_values(["prefix"])
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
display(recorddf)

,prefix,file
2,cuda_time,../data/cuda_time.log
0,ffma_flops,../data/ffma_flops.log
4,l1_atom,../data/l1_atom.log
3,l1_red,../data/l1_red.log
6,shared,../data/shared.log
1,tc_flops,../data/tc_flops.log
5,tc_time,../data/tc_time.log


In [124]:
resdf = pd.DataFrame(columns=resultkeys)

#metrics
for fin in recorddf["file"]:
#for fin in ['../data/time.log']:
    #project frame
    metricdf = pd.read_csv(fin)
    
    #fuse read/write metrics together:
    unique_metrics = metricdf["Metric Name"].unique()
    unique_metrics = set([x.replace(".sum","").replace("_write","").replace("_read","").replace("_ld","").replace("_st","") for x in unique_metrics])
    #add the metric type
    metricdf["Metric Type"] = "total"
    metricdf["Calls"] = 1
    #read
    metricdf.loc[ metricdf[ "Metric Name" ].str.contains("_read"), "Metric Type" ] = "read"
    metricdf.loc[ metricdf[ "Metric Name" ].str.contains("_ld"), "Metric Type" ] = "read"
    #write
    metricdf.loc[ metricdf[ "Metric Name" ].str.contains("_write"), "Metric Type" ] = "write"
    metricdf.loc[ metricdf[ "Metric Name" ].str.contains("_st"), "Metric Type" ] = "write"
    #rate
    metricdf.loc[ metricdf[ "Metric Name" ].str.contains(".per_second"), "Metric Type" ] = "rate"

    for metric in unique_metrics:
        metricdf.loc[ metricdf[ "Metric Name"].str.startswith(metric), "Metric Name" ] = metric
    #cleanups
    tmpdf = metricdf[["Kernel Name", "Calls", "Metric Name", "Metric Type", "Metric Value"]]
    #print(tmpdf)

    #compute the profile
    resdf = transpose_frame(resdf,tmpdf)
    #df_profiles.append(profiledf)

print(resdf)
#profiledf = pd.concat(df_profiles)
#profiledf.reset_index(drop=True, inplace=True)

                                          Kernel Name  Calls
0   Volta_hmma_implicit_gemm_fprop_fp32_nhwc_128x1...      2
1   Volta_hmma_implicit_gemm_fprop_fp32_nhwc_128x6...      2
2   Volta_hmma_implicit_gemm_fprop_fp32_nhwc_256x1...      4
3   Volta_hmma_implicit_gemm_wgrad_fp32_nhwc_128x1...      4
4   Volta_hmma_implicit_gemm_wgrad_fp32_nhwc_128x6...      1
..                                                ...    ...
91           volta_fp16_sgemm_fp16_32x32_sliced1x4_nt      1
92  volta_s884cudnn_fp16_128x128_ldg8_wgrad_exp_in...      1
93  volta_s884cudnn_fp16_128x128_ldg8_wgrad_idx_ex...      7
94  volta_s884cudnn_fp16_256x128_ldg8_wgrad_idx_ex...      5
95  volta_s884cudnn_fp16_64x64_sliced1x4_ldg8_wgra...      7

[96 rows x 2 columns]
                                          Kernel Name  Calls   CUDA Cycles  \
0   Volta_hmma_implicit_gemm_fprop_fp32_nhwc_128x1...      2  7.619451e+08   
1   Volta_hmma_implicit_gemm_fprop_fp32_nhwc_128x6...      2  8.214659e+07   
2   Volta_h

In [125]:
#display(profiledf.columns)
#tmplist = ['Name', 'Invocations', 'Pass', 'L1 Transactions Avg', 'L2 Transactions Avg', 'DRAM Transactions Avg']
#display(profiledf[tmplist])

# Compute AI Results

In [126]:
#profiledf[ (profiledf["Network Name"]=="ResNet50-2") &\
#           (profiledf["Input Shape"]=="112x112x64") &\
#           (profiledf["Batch Size"]==16) &\
#           (profiledf["Precision"]=="FP32") &\
#           (profiledf["Stride Size"]==2) &\
#           (profiledf["Pass"]=="forward") &\
#           (profiledf["Kernel Shape"]=="7x7x64x64")
#         ]
#profiledf

In [127]:
#sum over all kernels
combinedselectkeys = ["Precision", "Network Name", "Data Format", "Input Shape", "Kernel Shape", "Stride Size", \
                     "Batch Size", "Pass"]

#copy profiledf
combineddf = profiledf.copy()

#get the aggregated performance, including all kernels:
#compute weights: multiply all measures by the number of invocations
weighted = True
if weighted:
    #first, get all the names of metrics which need to be weighted
    metrics = [x for x in combineddf.columns if "Avg" in x]
    for metric in metrics:
        combineddf[metric] *= combineddf["Invocations"]

#sum up
combineddf = combineddf.groupby(by=combinedselectkeys).sum()#.reset_index()


#the flop fractions need to be recomputed
combineddf["FP32 FLOPs Fraction Avg"] = combineddf["FP32 FLOPs Avg"] / combineddf["FLOPs Avg"]
combineddf["FP16 FLOPs Fraction Avg"] = combineddf["FP16 FLOPs Avg"] / combineddf["FLOPs Avg"]
combineddf["TC FLOPs Fraction Avg"]   = combineddf["TC FLOPs Avg"]   / combineddf["FLOPs Avg"]

### Get performance
combineddf["Performance GFlop/s"]      = combineddf["FLOPs Avg"]      / (combineddf["CUDA Time Avg"]*10**9)
combineddf["FP32 Performance GFlop/s"] = combineddf["FP32 FLOPs Avg"] / (combineddf["CUDA Time Avg"]*10**9)
combineddf["FP16 Performance GFlop/s"] = combineddf["FP16 FLOPs Avg"] / (combineddf["CUDA Time Avg"]*10**9)
combineddf["TC Performance GFlop/s"]   = combineddf["TC FLOPs Avg"]   / (combineddf["TC Time Avg"]*10**9)


### Get AI
# L1
combineddf["L1 AI"]        = combineddf["FLOPs Avg"]      / combineddf["L1 Bytes Avg"]
combineddf["FP32 L1 AI"]   = combineddf["FP32 FLOPs Avg"] / combineddf["L1 Bytes Avg"]
combineddf["FP16 L1 AI"]   = combineddf["FP16 FLOPs Avg"] / combineddf["L1 Bytes Avg"]
combineddf["TC L1 AI"]     = combineddf["TC FLOPs Avg"]   / combineddf["L1 Bytes Avg"]
# L2
combineddf["L2 AI"]        = combineddf["FLOPs Avg"]      / combineddf["L2 Bytes Avg"]
combineddf["FP32 L2 AI"]   = combineddf["FP32 FLOPs Avg"] / combineddf["L2 Bytes Avg"]
combineddf["FP16 L2 AI"]   = combineddf["FP16 FLOPs Avg"] / combineddf["L2 Bytes Avg"]
combineddf["TC L2 AI"]     = combineddf["TC FLOPs Avg"]   / combineddf["L2 Bytes Avg"]
# DRAM
combineddf["DRAM AI"]      = combineddf["FLOPs Avg"]      / combineddf["DRAM Bytes Avg"]
combineddf["FP32 DRAM AI"] = combineddf["FP32 FLOPs Avg"] / combineddf["DRAM Bytes Avg"]
combineddf["FP16 DRAM AI"] = combineddf["FP16 FLOPs Avg"] / combineddf["DRAM Bytes Avg"]
combineddf["TC DRAM AI"]   = combineddf["TC FLOPs Avg"]   / combineddf["DRAM Bytes Avg"]

combineddf.sort_values(by=combinedselectkeys).reset_index(drop=True, inplace=True)

NameError: name 'profiledf' is not defined

In [ ]:
#display(combineddf.columns)
display(combineddf[['CUDA Time Avg', 'FP32 FLOPs Avg', 'FP16 FLOPs Avg', 'TC FLOPs Avg', 'FLOPs Avg', 'L1 Transactions Avg', 'L2 Transactions Avg', 'DRAM Transactions Avg']])
# combineddf.keys
# combineddf.columns

# combineddf['Name']
# combineddf.iloc[0,1]
# combineddf.iloc[2]

In [ ]:
#combineddf = combineddf.reset_index()
#seldf = combineddf[ (combineddf["Network Name"]=="ResNet50-2") &\
#           (combineddf["Input Shape"]=="112x112x64") &\
#           (combineddf["Precision"]=="FP32")]
#seldf
#combineddf[["FP32 L2 AI", "FP32 L1 AI"]]
combineddf[["L2 AI", "L1 AI", "DRAM AI", "FP32 DRAM AI", "FP16 DRAM AI"]]

# Export Data

In [ ]:
profiledf.to_csv(os.path.join(outputdir,"full_profile.csv"))
combineddf.to_csv(os.path.join(outputdir,"combined_profile.csv"))